<a href="https://colab.research.google.com/github/tristandewildt/ValueMonitor/blob/5570999/BERTopic_ValueMonitor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.9/103.9 KB 893.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 77.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.4/662.4 KB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 KB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 106.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

stop_words = stopwords.words('english')

def clean_text(x):
  x = str(x)
  x = x.lower()
  x = re.sub(r'#[A-Za-z0-9]*', ' ', x)
  x = re.sub(r'https*://.*', ' ', x)
  x = re.sub(r'@[A-Za-z0-9]+', ' ', x)
  tokens = word_tokenize(x)
  x = ' '.join([w for w in tokens if not w.lower() in stop_words])
  x = re.sub(r'[%s]' % re.escape('!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~“…”’'), ' ', x)
  x = re.sub(r'\d+', ' ', x)
  x = re.sub(r'\n+', ' ', x)
  x = re.sub(r'\s{2,}', ' ', x)
  return x

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
df_WOS = pd.read_csv("/content/drive/MyDrive/TU_Delft_DST/ValueMonitor_Data/WOS_hydrogen.csv")
df_WOS = df_WOS[df_WOS['AB'].notna()]
df_WOS.PY.head()
df_WOS['clean_text'] = df_WOS.AB.apply(clean_text)
date = df_WOS.PY.to_list()
abs = df_WOS.clean_text.to_list()

In [6]:
seed_topic_list = [["renewable energy", "peak", "storage", "power", "intermittent", "power generation", "electric power", "power system", "solar energy", "photovoltaic", "wind energy", "turbine"],
                   ["vehicle", "vehicles", "hydrogen cars", "cars", "transport", "transportation", "mobility"],
                   ["steel", "heating", "oil", "methanol", "refinery", "natural gas", "petrochemical", "chemical", "industry"]]

In [5]:
from bertopic import BERTopic
from umap import UMAP

umap_model = UMAP(n_neighbors=15, n_components=5, 
                  min_dist=0.0, metric='cosine', random_state=42)
# topic_model = BERTopic(umap_model=umap_model)

In [7]:
# from bertopic import BERTopic
topic_model = BERTopic(seed_topic_list=seed_topic_list, language="english", calculate_probabilities=True, verbose=True, umap_model=umap_model)
topics, probs = topic_model.fit_transform(abs)

Batches:   0%|          | 0/113 [00:00<?, ?it/s]

2023-02-10 11:08:53,881 - BERTopic - Transformed documents to Embeddings


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2023-02-10 11:09:23,138 - BERTopic - Reduced dimensionality
2023-02-10 11:09:23,991 - BERTopic - Clustered reduced embeddings


In [8]:
freq = topic_model.get_topic_info(); freq.head(5)

,Topic,Count,Name
0,-1,803,-1_hydrogen_energy_reaction_high
1,0,437,0_cells_stress_oxidative_ros
2,1,160,1_photocatalytic_light_photocatalysts_visible
3,2,133,2_stars_galaxies_mass_star
4,3,123,3_bond_calculations_hydrogen_energy


In [10]:
topic_model.get_topic(1)

[('photocatalytic', 0.046004634968801826),
 ('light', 0.026349253019408648),
 ('photocatalysts', 0.023757565839178558),
 ('visible', 0.023559941276903865),
 ('tio', 0.021094807267164013),
 ('cds', 0.02054306392578884),
 ('solar', 0.01786130074557334),
 ('charge', 0.01727194591630741),
 ('photocatalyst', 0.017135197665907962),
 ('band', 0.01653554040172844)]

In [11]:
topic_model.get_topic(2)

[('stars', 0.02864914699748439),
 ('galaxies', 0.022310179951185515),
 ('mass', 0.021287291137813653),
 ('star', 0.020512938896891663),
 ('line', 0.01966813127164768),
 ('gas', 0.017968620653268003),
 ('clouds', 0.015281498793277245),
 ('emission', 0.01482426728839995),
 ('similar', 0.014495109478826458),
 ('observations', 0.014429397861893189)]

In [12]:
topic_model.get_topic(18)

[('mg', 0.04467942932530648),
 ('hydrogen', 0.03611205934997943),
 ('storage', 0.03495071286325927),
 ('ni', 0.034013135126608786),
 ('mgh', 0.03092041178645901),
 ('alloys', 0.030649047410975586),
 ('la', 0.023136500631057683),
 ('magnesium', 0.021717913640251173),
 ('temperature', 0.020914131900957564),
 ('phase', 0.020821143810885935)]

In [9]:
topic_model.visualize_topics()